# Crix11 - Dream11 AI Agent for IPL 2025

This notebook demonstrates how to use the Dream11 AI Agent to create optimized fantasy cricket teams for IPL 2025.

In [ ]:
import os
import datetime
import warnings
from dotenv import load_dotenv
from IPython.display import Markdown, display

# Suppress warnings
warnings.filterwarnings("ignore")

# Load environment variables
load_dotenv()

## Step 1: Check Environment Variables

First, let's make sure our environment variables are set correctly.

In [ ]:
# Check environment variables (don't show the actual keys)
env_vars = {
    "OPENAI_API_KEY": bool(os.getenv("OPENAI_API_KEY")),
    "SERPER_API_KEY": bool(os.getenv("SERPER_API_KEY")),
    "MATCH_URL": os.getenv("MATCH_URL"),
    "PLAYER_STATS_URL": os.getenv("PLAYER_STATS_URL")
}

print("Environment Variables:")
for key, value in env_vars.items():
    print(f"- {key}: {'✅ Set' if value else '❌ Not Set'}")

## Step 2: Import Agent Modules

In [ ]:
from src.agents.todays_teams_agent import create_todays_teams_agent, create_todays_teams_task
from src.agents.user_team_selector_agent import create_user_team_selector_agent, create_user_team_selector_task
from src.agents.players_agent import create_players_agent, create_players_task
from src.agents.team_selector_agent import create_team_selector_agent, create_team_selector_task
from crewai import Crew, Process

## Step 3: Set Up Agents and Tasks

In [ ]:
# Get current date
current_date = datetime.datetime.now().strftime("%Y-%m-%d")
print(f"Current Date: {current_date}")

# Get environment variables
match_url = os.getenv("MATCH_URL")
player_stats_url = os.getenv("PLAYER_STATS_URL")

# Create agents
todays_teams_agent = create_todays_teams_agent()
user_team_selector_agent = create_user_team_selector_agent()
players_agent = create_players_agent()
team_selector_agent = create_team_selector_agent()

# Create tasks
todays_teams_task = create_todays_teams_task(todays_teams_agent, current_date, match_url)
user_team_selector_task = create_user_team_selector_task(user_team_selector_agent, todays_teams_task)
players_task = create_players_task(players_agent, user_team_selector_task, player_stats_url)
select_players_task = create_team_selector_task(team_selector_agent, players_task)

## Step 4: Run Today's Teams Agent First

Let's first run just the Today's Teams agent to see what matches are available today.

In [ ]:
# Create a crew with just the today's teams task
todays_matches_crew = Crew(
    agents=[todays_teams_agent],
    tasks=[todays_teams_task],
    verbose=True
)

# Run the crew
todays_matches_result = todays_matches_crew.kickoff()

# Display results
display(Markdown("## Today's Matches"))
display(Markdown(todays_matches_result))

## Step 5: Run the Full Dream11 Team Selection Process

In [ ]:
# Create crew for team selection
team_crew = Crew(
    agents=[user_team_selector_agent, players_agent, team_selector_agent],
    tasks=[user_team_selector_task, players_task, select_players_task],
    process=Process.sequential,
    verbose=True
)

# Run the crew
team_result = team_crew.kickoff()

# Display results
display(Markdown("## Dream11 Team Selection"))
display(Markdown(team_result))

## Potential Improvements

The Dream11 Team Selection System could be improved in several ways:

1. **Real-Time Player Credit Details**: Integrate with Dream11 API for accurate credit values
2. **RAG-Based System**: Implement a Retrieval-Augmented Generation approach with a database of IPL stats
3. **Post-Toss Execution**: Run the system after the toss for confirmed playing 11
4. **Enhanced User Customization**: Allow users more control over player selection